## Pablo Valdunciel Sánchez 
### 3rd November, 2019

Implement the DBCluster algorthim

# Imports

In [7]:
import numpy as np 
import matplotlib.pyplot as plt
import itertools

from sklearn.datasets import load_iris
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.metrics import accuracy_score

from scipy.cluster.hierarchy import dendrogram 
from scipy.stats import mode

#  DBCluster class

In [2]:
class DBCluster:
    """DBCluster class"""


    def __init__(self, n_clusters, D):
        """ Initializes the DBCluster instance """
        self.n_clusters = n_clusters
        self._D = D.copy()
        self._current_partition = [np.array(range(D.shape[0]))] #Po
        self._partition_history = []
        print(self)
    
    @staticmethod
    def _get_indexes(target, myList):
        """ Returns the indexes where the list 'myList' contains the 
            specified element 'target'
        """
        for i in range(len(myList)):
            if myList[i] == target:
                yield i
                
            
    def __str__(self):
        """ Provides a text representation of the DBCluster instance """
        return "DBCluster(n_clusters={})".format(self._n_clusters)
    
    def get_partition_history(self):
        """ Gets the partition history """
        return self._partition_history.copy() 
    
    def _geometric_var(self, cluster):
        """ Obtains the geometric variability of a subset of points """
        m = len(cluster)        
        g_var = 0 

        for i in range(len(cluster)):
            for j in range(i,len(cluster)):
                g_var += (self._D[i][j]**2)

        return g_var /(2*(m**2))
    
    def divide(self):
        """ DBCluster algorithm """
        
        while len(self._current_partition) < self._n_clusters:
                        
            # Obtain all possible divisions
            all_divisions = self._get_possible_divisions()
            g_var = np.empty(len(all_divisions), dtype=float)
            
            # Compute the geometric variability of each possible division
            for i, d in enumerate(all_divisions): 
                cluster = self._current_partition[d["cluster"]]
                subcluster_1 = np.array(cluster[d["subcluster1"]])
                subcluster_2 = np.array(cluster[d["subcluster2"]])
                g_var[i] = self._geometric_var(subcluster_1) + self._geometric_var(subcluster_2)
                
            # Get the optimal division (minimal geometric variability)
            opt_division = all_divisions[np.argmin(g_var)]
            cluster = self._current_partition[opt_division["cluster"]]
            new_cluster_1 = np.array(cluster[opt_division["subcluster1"]])
            new_cluster_2 = np.array(cluster[opt_division["subcluster2"]])            
            
            # Add the new partition to the history
            self._partition_history.append(self._current_partition[:])
            
            # Modify the partition
            del self._current_partition[opt_division["cluster"]]
            self._current_partition.append(new_cluster_1)
            self._current_partition.append(new_cluster_2)
        
        self._partition_history.append(self._current_partition[:])
                       
    
    def _get_possible_divisions(self):  
        """ Generates all possible divisions given the current partition """
        all_divisions = []
        
        for index, cluster in enumerate(self._current_partition):
            binary_combinations = list(itertools.product([0, 1], repeat=len(cluster)))

            for comb in binary_combinations:
                if len(np.unique(comb)) > 1:
                    zero_indexes = list(DBCluster._get_indexes(0, list(comb)))
                    one_indexes = list(DBCluster._get_indexes(1, list(comb))) 
                    division = {"cluster":index, "subcluster1":np.array(zero_indexes), "subcluster2":np.array(one_indexes)}
                    all_divisions.append(division)

        return all_divisions
    

### Test

In [193]:
D = np.array([ 
    [0, 1, 3, 4, 7],
    [1, 0, 4, 4, 8],
    [3, 4, 0, 2, 8],
    [4, 4, 2, 0, 7],
    [7, 8, 8, 7, 0]
])

In [194]:
db = DBCluster(5, D)
db.divide()

DBCluster(n_clusters=5)


In [195]:
db.get_partition_history()

[[array([0, 1, 2, 3, 4])],
 [array([0, 1, 2]), array([3, 4])],
 [array([0, 1, 2]), array([3]), array([4])],
 [array([3]), array([4]), array([0, 1]), array([2])],
 [array([3]), array([4]), array([2]), array([0]), array([1])]]